In [5]:
from MOF_build import buildUIO66
from MDprepare import move2C
from MDprepare import ff_gen as ff
from MDprepare import itp_process as ip
import numpy as np

In [6]:
'''
#STEP1 Build a MOF model 
'''

linker_pdb = "../linkers4demo/66.pdb"
x_num, y_num, z_num, x_scalar, y_scalar, z_scalar = 1,1,1,14.5,14.5,14.5
outnode, outlinker, outcut, outall = ("66node", "66linker", "66cut", "66all")  # name of output file
O1_index, O2_index, O3_index = 9,11,10  ##NOTE:O1->O2//short edge O-O ,O1->O3//long edge O-O

In [7]:
mof = buildUIO66.UIO66(linker_pdb, O1_index, O2_index, O3_index, x_num, y_num, z_num,x_scalar, y_scalar, z_scalar )
mof.buildall(outnode, outcut, outlinker,outall)
mof.view(outall)
#move atoms to box center and reset box size 
move2C.atoms2c('66all',0.9,1.6,0.9,5,3.5,4.3) 


points number:  8
linkers number:  18
total atoms: 1236
ZRR   48
OMM   32
OHH   32
CUT   60
UNK   18

Time cost (s):   1:52:54.186335


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
'''
#STEP2 generate linker forcefield 
'''


linkerpath='Residues'
basis = 'sto-3g'
charge = -2
grofile_name,residue_name ='cph','UNK'
ff.ff_gen(linkerpath,charge,basis,grofile_name,residue_name)


* Info * Reading basis set from file: /Users/chenxili/miniconda/envs/vlx/lib/python3.11/site-packages/veloxchem/basis/STO-3G
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: STO-3G                                                                              
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          
              

In [9]:
"""step3 generate topology file for MD simulation"""


middlelines, sectorname = ip.parsetop("MDdemo/itp/template.top")
res_num = (
    "ZRR            48\nOMM            32\nOHH            32\nCUT             60\n"
    + "UNK            18\nSOL            1700\nK            10\n"
)
ip.top_gen("itps/cph.itp", middlelines, "em.top", res_num)

In [10]:
''' 
insert 1000 water molecules in the box before MD running
'''
a = 'MDdemo/mdfiles'
mdp = 'MDdemo/mdp'
!gmx insert-molecules -f new66all.gro -ci MDdemo/itp/tip3p.gro -nmol 1700 -o $a/mof_sol.gro -scale 0.4
!gmx insert-molecules -f $a/mof_sol.gro -ci MDdemo/itp/K.gro -nmol 10 -o $a/em.gro  -scale 0.4

            :-) GROMACS - gmx insert-molecules, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/chenxili/GitHub/MOF_build_Demo/mof_build_demo
Command line:
  gmx insert-molecules -f new66all.gro -ci MDdemo/itp/tip3p.gro -nmol 1700 -o MDdemo/mdfiles/mof_sol.gro -scale 0.4

Reading solute configuration
Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This 

In [11]:
''' run MD simulation ''' 
!gmx grompp -f mdfiles/mdp/em.mdp   -c $a/em.gro    -p $a/em.top   -o $a/em.tpr -maxwarn 1 
!gmx mdrun -s $a/em.tpr -deffnm $a/em_out -g $a/em.log
#!gmx grompp -f $mdp/nvt.mdp -c $a/em_out.gro -p $a/em.top -o $a/nvt.tpr -maxwarn 1 
#!gmx mdrun  -s $a/nvt.tpr  -deffnm $a/nvt_out -g $a/nvt.log
#!gmx grompp  -f $mdp/npt.mdp -c $a/nvt_out.gro -p $a/em.top -o $a/npt.tpr -maxwarn 1 
#!gmx mdrun  -s $a/npt.tpr  -deffnm $a/npt_out -g $a/npt.log
#!gmx grompp -f $mdp/md.mdp -c $a/npt_out.gro  -p $a/em.top -o $a/md.tpr -maxwarn 1 
#!gmx mdrun  -s $a/md.tpr  -deffnm $a/md_out -g $a/md.log

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/chenxili/GitHub/MOF_build_Demo/mof_build_demo
Command line:
  gmx grompp -f mdfiles/mdp/em.mdp -c MDdemo/mdfiles/em.gro -p MDdemo/mdfiles/em.top -o MDdemo/mdfiles/em.tpr -maxwarn 1


-------------------------------------------------------
Program:     gmx grompp, version 2024.2-Homebrew
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int*, char**)

Error in user input:
Invalid command-line options
  In command-line option -f
    File 'mdfiles/mdp/em.mdp' does not exist or is not accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 2)

For more information and tips for troubleshooting, please check the GROMACS
website at https://manual.gromacs.

In [12]:
!gmx grompp -f $mdp/nvt.mdp -c $a/em_out.gro -p $a/em.top -o $a/nvt.tpr -maxwarn 1 
!gmx mdrun  -s $a/nvt.tpr  -deffnm $a/nvt_out -g $a/nvt.log

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/chenxili/GitHub/MOF_build_Demo/mof_build_demo
Command line:
  gmx grompp -f MDdemo/mdp/nvt.mdp -c MDdemo/mdfiles/em_out.gro -p MDdemo/mdfiles/em.top -o MDdemo/mdfiles/nvt.tpr -maxwarn 1


-------------------------------------------------------
Program:     gmx grompp, version 2024.2-Homebrew
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int*, char**)

Error in user input:
Invalid command-line options
  In command-line option -c
    File 'MDdemo/mdfiles/em_out.gro' does not exist or is not accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 2)

For more information and tips for troubleshooting, please check the GROMACS
website at https://man

In [13]:
!gmx grompp  -f $mdp/npt.mdp -c $a/nvt_out.gro -p $a/em.top -o $a/npt.tpr -maxwarn 1 
!gmx mdrun  -s $a/npt.tpr  -deffnm $a/npt_out -g $a/npt.log

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/chenxili/GitHub/MOF_build_Demo/mof_build_demo
Command line:
  gmx grompp -f MDdemo/mdp/npt.mdp -c MDdemo/mdfiles/nvt_out.gro -p MDdemo/mdfiles/em.top -o MDdemo/mdfiles/npt.tpr -maxwarn 1


-------------------------------------------------------
Program:     gmx grompp, version 2024.2-Homebrew
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int*, char**)

Error in user input:
Invalid command-line options
  In command-line option -c
    File 'MDdemo/mdfiles/nvt_out.gro' does not exist or is not accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 2)

For more information and tips for troubleshooting, please check the GROMACS
website at https://m